# Google Colab Admin

In [ ]:
!pip install PyDrive
!pip install keras --upgrade
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

In [ ]:
# authorise google SDK to use drive

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# from the shareable link on google, get the id and paste like this

# get annotations file
download = drive.CreateFile({'id':'1XHHRjyRyq1wNnmaP6ZRn7c12qID9J-M9'})
download.GetContentFile('Annotations.zip')

# get signals file
download = drive.CreateFile({'id':'17JL1xg5wNnl8ck8VqYsvpABxdAIrdlBi'})
download.GetContentFile('Signals.zip')

In [ ]:
# open the zipfiles

import zipfile

zip_ref = zipfile.ZipFile('Annotations.zip', 'r')
zip_ref.extractall()
zip_ref.close()

zip_ref = zipfile.ZipFile('Signals.zip', 'r')
zip_ref.extractall()
zip_ref.close()

# Importing Audio Files

In [ ]:
''' IMPORT AUDIO SIGNALS '''

# Write signal data to variables
from scipy.io import wavfile
import wave
import numpy as np
samprate = 44100

def get_wav(file_name, nsamples):
    wav = wavfile.read(file_name)[1]
    signal = wav[0:nsamples]
    return signal

filepath = "/users/garethjones/Documents/Data Science/Feebris/Data/Clean Stethoscope/"

# work out length of each sample in frames
sampleframes = []
for i in range(1,11):
    sample = wave.open(filepath+"Signals/Signal {}.wav".format(i),'r')
    nframes = sample.getnframes()
    sampleframes.append(nframes)
    
# import signals and store
signals = [get_wav(filepath+"Signals/Signal {}.wav".format(i+1),sampleframes[i]*2) for i in range(10)]
    
# create list of second intervals for each sample
samplesecs = [np.arange(1,len(signals[i])+1) / samprate for i in range(10)]

In [ ]:
''' IMPORT ANNOTATION AUDIO SIGNALS '''

# work out length of each sample in frames
annotationframes = []
for i in range(1,11):
    sample = wave.open(filepath+"Annotations/Annotation {}.wav".format(i),'r')
    nframes = sample.getnframes()
    annotationframes.append(nframes)
    
# import signals and store
annotations = [get_wav(filepath+"Annotations/Annotation {}.wav".format(i+1),annotationframes[i]*2) for i in range(10)]

In [ ]:
''' NORMALIZE SIGNALS TO 1 '''

# Normalize signals
for i in range(len(signals)):
    signals[i] = signals[i] / (2.**15)

for i in range(len(annotations)):
    annotations[i] = annotations[i] / (2.**15)  

In [ ]:
''' CUT OUT SIGNALS WHERE ANNOTATIONS ARE NOT CLEAR '''

# set all signals to be the length of annotations
for i in range(len(signals)):
    signals[i] = signals[i][0:len(annotations[i])]

# pre-process signals 1,2,7 specifically due to poor click annotations
signals[0] = signals[0][0:2700000]
annotations[0] = annotations[0][0:2700000]

signals[1] = signals[1][0:2650000]
annotations[1] = annotations[1][0:2650000]

signals[6] = signals[6][0:2800000]
annotations[6] = annotations[6][0:2800000]

# test to ensure lengths are the same
for i in range(len(signals)):
    assert len(signals[i]) == len(annotations[i])

In [ ]:
''' CUT SIGNALS TO SHORTEST LENGTH (FOR EQUAL SIZED SPECTOGRAMS) '''

# to ensure spectograms work well, I need to cut the data at 2,650,000, the shortest sample length, so everything is same length
# get all signal lengths
signallen = [len(signals[i]) for i in range(len(signals))] 

for i in range(len(signals)):
    signals[i] = signals[i][0:min(signallen)]

for i in range(len(signals)):
    annotations[i] = annotations[i][0:min(signallen)]

In [ ]:
''' VISUALISE '''

import matplotlib.ticker as ticker
fig,ax = plt.subplots(1,1,figsize=(20,8))
ax.plot(annotations[6])
ax.xaxis.set_major_locator(ticker.MultipleLocator(125000))

# Build Mel Spectrograms

In [ ]:
''' IMPORTS & GLOBALS '''

import matplotlib.pyplot as plt
import librosa
from librosa.display import specshow
from librosa.feature import melspectrogram
import pylab
import warnings
warnings.filterwarnings('ignore')
samprate = 44100

In [ ]:
''' MAKE SIGNALS MONO AUDIO '''

# select only one channel of stereo signal, and transpose ready for melspectogram
signals_mono = [signals[i].T[0] for i in range(len(signals))]

In [ ]:
''' GENERATE MEL SPECTOGRAMS FOR SIGNALS '''

hop_length = 2550  # number of frames to jump when computing fft
fmin = 125  # bottom frequency to look at
fmax = 500  # top frequency to look at
n_mels = 45  # number of audio frequency bins
n_fft = [8000, 11025, 14000]  # width of the fft windows


# list of 10 mels, with depth 3
mel_db_list = []

for i in range(len(signals_mono)):
    
    mel = [melspectrogram(
            signals_mono[i],
            sr = samprate,
            hop_length = 2550,
            n_fft = j,
            n_mels = 45,
            fmin = 125,
            fmax = 500) for j in n_fft]
    
    mel_db = [librosa.power_to_db(mel[k],ref=np.max) for k in range(len(mel))]
    
    mel_db = np.stack(mel_db,axis=-1)
    
    mel_db_list.append(mel_db)

# Generate Annotation Gates

In [ ]:
''' GLOBAL VARIABLES & REFERENCE '''

# known clicks per sample
originalclickspersignal = {
        'Annotation 1' : 24, 'Annotation 2' : 20, 'Annotation 3' : 24, 'Annotation 4' : 34, 'Annotation 5' : 30,
        'Annotation 6' : 31, 'Annotation 7' : 32, 'Annotation 8' : 36, 'Annotation 9' : 33, 'Annotation 10' : 29
        }

# these are number of clicks per sample after chopping data
newclickspersignal = {
        'Annotation 1' : 22, 'Annotation 2' : 19, 'Annotation 3' : 22, 'Annotation 4' : 33, 'Annotation 5' : 28,
        'Annotation 6' : 30, 'Annotation 7' : 31, 'Annotation 8' : 29, 'Annotation 9' : 29, 'Annotation 10' : 26
        }

# eye-balled thresholds for each signal
thresholds = {
        'Annotation 1' : 0.1, 'Annotation 2' : 0.1, 'Annotation 3' : 0.07, 'Annotation 4' : 0.1, 'Annotation 5' : 0.03,
        'Annotation 6' : 0.1, 'Annotation 7' : 0.095, 'Annotation 8' : 0.1, 'Annotation 9' : 0.05, 'Annotation 10' : 0.1
        }

In [ ]:
''' MAKE ANNOTATION SIGNALS MONO '''

# make annotations one channel, transposed, absolute
annotations_mono = []
for i in range(len(annotations)):
    x = abs(annotations[i].T[0])
    annotations_mono.append(x)

In [ ]:
''' TURN SIGNALS INTO STEP FUNCTIONS '''

# denote whenever amplitude is above threshold
anno_gates = []
for i in range(len(annotations_mono)):
    gate_list = []
    for j in range(len(annotations_mono[i])):
        if annotations_mono[i][j] > thresholds['Annotation {}'.format(i+1)]: # this is amplitude threshold
            x = 1
        else:
            x = 0
        gate_list.append(x)
    anno_gates.append(gate_list)

In [ ]:
''' SUPRESS NOISE IN STEP FUNCTION '''

# ensure noise is removed so there's exact number of clicks
fwd_frame_thresh = 15000
size_anno_gates = []
for i in range(len(anno_gates)):
    for j in range(len(anno_gates[i])):
        if anno_gates[i][j] == 1:
            for k in range(1,fwd_frame_thresh): # this is the forward threshold for silencing frames
                if j+k < len(anno_gates[i]):
                    anno_gates[i][j+k] = 0
                else:
                    k = fwd_frame_thresh - j
                    anno_gates[i][j+k] = 0       
    size_anno_gates.append(sum(anno_gates[i]))

print(np.r_[list(newclickspersignal.values())] - np.r_[size_anno_gates])

In [ ]:
''' REDUCE ANNOTATIONS TO LENGTH OF MEL SPECTROGRAM '''

# set the search window and hop size to same as the melspectrogram
hop_size = 2550
search_window = 11025

# find index points to look at in annotation signal
indices = list(np.arange(0,len(anno_gates[0]),2550))

# now look across the annotation signal and max if there's a 1 in the window frame
labels_list = []
for i in range(len(anno_gates)):
    labels = []
    
    for j in indices:    
        if ((j - search_window/2) > 0) & ((j + search_window/2) < len(anno_gates[i])):
            label_window = anno_gates[i][int(j-search_window/2):int(j+search_window/2)]
            max_label = max(label_window)
            labels.append(max_label)
        
        elif (j - search_window/2) < 0:
            label_window = anno_gates[i][0:int(j+search_window/2)]
            max_label = max(label_window)
            labels.append(max_label)
        
        elif (j + search_window/2) > len(anno_gates[i]):
            label_window = anno_gates[i][int(j-search_window/2):len(anno_gates[i])]
            max_label = max(label_window)
            labels.append(max_label)
    
    labels_list.append(labels)

In [ ]:
''' VISUALISE '''
fig,axs = plt.subplots(2,1,figsize=(12,8))
axs[0].plot(labels_list[2])
axs[0].set_title('New Labels (1040 Frames)')
axs[1].plot(anno_gates[2])
axs[1].set_title('Original Annotation Signal')
plt.show()
plt.close()

# Split and Prep Data for Model

In [ ]:
''' NEED SOMEONE TO CHECK THIS CODE FOR ME '''

''' CHOP SPECTOGRAMS AND NORMALIZE '''

from sklearn.preprocessing import scale

window_len = 15

mel_slices_normed_list = []
for i in range(len(mel_db_list)):
    
    for j in range(mel_db.shape[1]-15):
        slices = mel_db_list[i][:,j:j+15,:]
        # normalize to zero mean and unit variance along rows of each spectogram within each slice
        # consider looking at this along columns as well, may improve training?
        slices_normed = [scale(slices[:,:,k],axis=1) for k in range(3)]
        # stacks elements of list back into a single numpy array
        slices_normed = np.stack(slices_normed,axis=-1)
        # puts theses single arrays into a list, length of one signal
        mel_slices_normed_list.append(slices_normed)

In [ ]:
''' CREATE NEW LABELS FOR EACH FRAME '''

# looks for the labels at the middle value of each mel spectrogram slice of window size 15
labels_list_shrunk = []
for i in range(len(labels_list)):

    for j in range(int(window_len/2),mel_db.shape[1]-int(window_len/2)-1):
        
        if labels_list[i][j] == 1:
            x = 1
        else:
            x = 0
        
        labels_list_shrunk.append(x)

In [ ]:
''' COMPARE OLD AND NEW LABELS '''

labels_sums = [sum(labels_list[i]) for i in range(len(labels_list))]
labels_shrunk_sums = [sum(labels_list_shrunk[i*1025:(i+1)*1025]) for i in range(len(labels_list))]
print('Original positive labels = ' + str(labels_sums))
print('New shrunk positive labels = ' + str(labels_shrunk_sums))

In [ ]:
''' CHOOSE RANDOM SLICES AND LABELS FOR MODELLING '''
# split data into training 70%, validation 20%, test 10%

indices = np.arange(0,len(mel_slices_normed_list))
np.random.shuffle(indices)

indices_train = indices[0:int(0.7*len(mel_slices_normed_list))]
indices_val = indices[int(0.7*len(mel_slices_normed_list)):int(0.9*len(mel_slices_normed_list))]
indices_test = indices[int(0.9*len(mel_slices_normed_list)):len(mel_slices_normed_list)]

assert len(indices_test)+len(indices_train)+len(indices_val) == len(mel_slices_normed_list)


mel_slices_train = np.array([mel_slices_normed_list[i] for i in indices_train])
labels_train = np.array([labels_list_shrunk[i] for i in indices_train])

mel_slices_val = np.array([mel_slices_normed_list[i] for i in indices_val])
labels_val = np.array([labels_list_shrunk[i] for i in indices_val])

mel_slices_test = np.array([mel_slices_normed_list[i] for i in indices_test])
labels_test = np.array([labels_list_shrunk[i] for i in indices_test])

assert len(mel_slices_train) == len(labels_train)
assert len(mel_slices_val) == len(labels_val)
assert len(mel_slices_test) == len(labels_test)

# Build 2D Conv Model

In [ ]:
''' DEFINE MODEL '''

from keras import models,layers,optimizers

model = models.Sequential()
model.add(layers.Conv2D(32,(3,7),activation='relu',input_shape=(45,15,3)))
model.add(layers.MaxPooling2D((3,1)))
model.add(layers.Conv2D(64, (3,3), activation="relu"))
model.add(layers.MaxPooling2D((3,1)))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation="relu"))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(1, activation="sigmoid"))
model.summary()

In [ ]:
''' COMPILE MODEL '''

model.compile(
    optimizer=optimizers.RMSprop(lr=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy"])

In [ ]:
''' TRAIN MODEL '''

history = model.fit(
        mel_slices_train,
        labels_train,
        batch_size=32,
        epochs = 100,
        validation_data = (mel_slices_val,labels_val),
        verbose=1)